In [1]:
# Load the autoreload extension
%load_ext autoreload

# Automatically reload modules before executing code
%autoreload 2

In [22]:
from pathlib import Path
from benchmark_src.results_processing.plots.plot_utils import get_task_df, get_list_of_all_runs
from benchmark_src.results_processing import ranking
from benchmark_src.results_processing.plots import table_retrieval_plots
import pandas as pd
import numpy as np

In [3]:
results_folder = Path("../../results_main")
assert results_folder.exists(), f"Could not find results folder at {results_folder}"

# Column Similarity Search results table

In [4]:
task_df = get_task_df(results_folder=results_folder, task_name="column_similarity_search")

Unique datasets (column_similarity_search): 5


In [5]:
task_df.groupby(['task', 'Approach', 'Configuration'])['dataset'].nunique().reset_index(name='num_datasets')

,task,Approach,Configuration,num_datasets
0,column_similarity_search,GritLM,"embedding_model=GritLM_GritLM-7B,table_row_lim...",5
1,column_similarity_search,hytrel,hytrel,5
2,column_similarity_search,sap_rpt_oss,"bagging=1,max_context_size=2048,predML_based_o...",5
3,column_similarity_search,sentence_transformer,"embedding_model=BAAI_bge-base-en-v1.5,table_ro...",4
4,column_similarity_search,sentence_transformer,"embedding_model=all-MiniLM-L6-v2,table_row_lim...",5
5,column_similarity_search,sentence_transformer,embedding_model=ibm-granite_granite-embedding-...,4
6,column_similarity_search,sentence_transformer,embedding_model=ibm-granite_granite-embedding-...,4
7,column_similarity_search,tabicl,"n_estimators=32,predML_based_on=custom_predict...",5


## Configure which approaches to include

In [6]:
# get list as ouptput and copy it to the next cell to comment out approaches not to be plotted
get_list_of_all_runs(task_df)

include_runs = [
    ('GritLM', 'embedding_model=GritLM_GritLM-7B,table_row_limit=100'),
    ('hytrel', 'hytrel'),
    ('sap_rpt_oss', 'bagging=1,max_context_size=2048,predML_based_on=custom_predictiveML_model'),
    ('sentence_transformer', 'embedding_model=BAAI_bge-base-en-v1.5,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=all-MiniLM-L6-v2,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-30m-english,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-english-r2,table_row_limit=100'),
    ('tabicl', 'n_estimators=32,predML_based_on=custom_predictiveML_model'),
]


In [7]:
include_runs = [
    #('GritLM', 'embedding_model=GritLM_GritLM-7B,table_row_limit=100'),
    ('hytrel', 'hytrel'),
    ('sap_rpt_oss', 'bagging=1,max_context_size=2048,predML_based_on=custom_predictiveML_model'),
    #('sentence_transformer', 'embedding_model=BAAI_bge-base-en-v1.5,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=all-MiniLM-L6-v2,table_row_limit=100'),
    #('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-30m-english,table_row_limit=100'),
    #('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-english-r2,table_row_limit=100'),
    ('tabicl', 'n_estimators=32,predML_based_on=custom_predictiveML_model'),
]

In [13]:
# Use the list to filter the original task_df
filtered_task_df = task_df[
    task_df.set_index(['Approach', 'Configuration']).index.isin(include_runs)
]

In [14]:
filtered_task_df.columns

Index(['Approach', 'Configuration', 'task', 'dataset', '# Runs', 'MRR_mean',
       'MAP_mean', 'Precision_mean', 'Recall_mean', 'MRR_std_mean',
       'MAP_std_mean', 'Precision_std_mean', 'Recall_std_mean', '_rows_count',
       'model_setup---execution_time (s)', 'model_setup---peak_cpu (%)',
       'model_setup---average_cpu (%)', 'model_setup---peak_memory (MB)',
       'model_setup---average_memory (MB)',
       'task_inference---execution_time (s)', 'task_inference---peak_cpu (%)',
       'task_inference---average_cpu (%)', 'task_inference---peak_memory (MB)',
       'task_inference---average_memory (MB)'],
      dtype='object')

In [15]:
# only keep columns 'Approach', 'Configuration', 'task', 'dataset', 'MRR_mean', 'MRR_std_mean', 'MAP_mean', 'MAP_std_mean', 'Precision_mean', 'Precision_std_mean', 'Recall_mean', 'Recall_std_mean', '# Runs'
filtered_task_df = filtered_task_df[['Approach', 'Configuration', 'task', 'dataset',  "MRR_mean", 'MRR_std_mean', 'MAP_mean', 'MAP_std_mean', 'Precision_mean', 'Precision_std_mean', 'Recall_mean', 'Recall_std_mean', '# Runs']]   

In [16]:
#filtered_task_df

In [17]:
# create results table, dataset names as rows, approaches as columns (order is hytrel, sap_rpt_oss, sentence_transformer, tabicl)

# pivot the dataframe to have datasets as rows and approaches as columns
results_table = filtered_task_df.pivot_table(
    index='dataset',
    columns=['Approach', 'Configuration'],
    values='MRR_mean',
    aggfunc='mean'
)

# flatten the multi-level columns
results_table.columns = [f"{approach} ({config})" for approach, config in results_table.columns]

# row order should be nextia, valentine, opendata, wikijoin-small, autojoin
results_table = results_table.reindex(['nextia', 'valentine', 'opendata', 'wikijoin_small', 'autojoin'])

# rename columns (order is hytrel, sap_rpt_oss, sentence_transformer, tabicl)
results_table.columns = ['hytrel', 'sap rpt oss', 'sentencetransformer', 'tabicl']

In [18]:
# add another empty column "our model" at the end
results_table['our model'] = "" 

In [19]:
results_table

,hytrel,sap rpt oss,sentencetransformer,tabicl,our model
dataset,,,,,
nextia,0.364130,0.005435,0.391304,0.000000,
valentine,0.264722,0.000099,0.585435,0.013523,
opendata,0.429365,0.199830,0.570465,0.000000,
wikijoin_small,0.743206,0.192079,0.939667,0.000000,
autojoin,0.000000,0.000000,0.000000,0.000000,


In [20]:
# get results_table as latex with formatting (two decimal places) and output written to file so that linebreaks are automatically handled, also add | between all columns
with open("column_similarity_search_results_table.tex", "w") as f:
    f.write(results_table.to_latex(float_format="%.2f", column_format="|l|" + "c|" * (len(results_table.columns))))

# Aggregated results overview table

In [ ]:
# use this information to get the correct metrics
"predictive_ml": [
    "LinearRegression_rmse (↓)",
    "XGBoost_roc_auc_score (↑)",
    "XGBoost_log_loss (↓)"
],

In [17]:
predictive_ml_df = get_task_df(results_folder=results_folder, task_name="predictive_ml")
row_similarity_search_df = get_task_df(results_folder=results_folder, task_name="row_similarity_search")
column_similarity_search_df = get_task_df(results_folder=results_folder, task_name="column_similarity_search")

Unique datasets (predictive_ml): 51
Unique datasets (row_similarity_search): 9
Unique datasets (column_similarity_search): 5


In [30]:
# get list as ouptput and copy it to the next cell to comment out approaches not to be plotted
get_list_of_all_runs(predictive_ml_df)

include_runs = [
    ('GritLM', 'embedding_model=GritLM_GritLM-7B,table_row_limit=100'),
    ('baseline', 'baseline'),
    ('hytrel', 'hytrel'),
    ('sap_rpt_oss', 'bagging=1,max_context_size=2048,predML_based_on=custom_predictiveML_model'),
    ('sap_rpt_oss', 'bagging=1,max_context_size=2048,predML_based_on=row_embeddings'),
    ('sentence_transformer', 'embedding_model=BAAI_bge-base-en-v1.5,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=all-MiniLM-L6-v2,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-30m-english,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-english-r2,table_row_limit=100'),
    ('tabicl', 'n_estimators=32,predML_based_on=custom_predictiveML_model'),
    ('tabicl', 'n_estimators=32,predML_based_on=row_embeddings'),
    ('tabpfn', 'device=cuda,predML_based_on=custom_predictiveML_model'),
    ('tabpfn', 'device=cuda,predML_based_on=row_embeddin

In [31]:
include_runs = [
    #('GritLM', 'embedding_model=GritLM_GritLM-7B,table_row_limit=100'),
    ('baseline', 'baseline'),
    ('hytrel', 'hytrel'),
    ('sap_rpt_oss', 'bagging=1,max_context_size=2048,predML_based_on=custom_predictiveML_model'),
    #('sap_rpt_oss', 'bagging=1,max_context_size=2048,predML_based_on=row_embeddings'),
    #('sentence_transformer', 'embedding_model=BAAI_bge-base-en-v1.5,table_row_limit=100'),
    ('sentence_transformer', 'embedding_model=all-MiniLM-L6-v2,table_row_limit=100'),
    #('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-30m-english,table_row_limit=100'),
    #('sentence_transformer', 'embedding_model=ibm-granite_granite-embedding-english-r2,table_row_limit=100'),
    ('tabicl', 'n_estimators=32,predML_based_on=custom_predictiveML_model'),
    #('tabicl', 'n_estimators=32,predML_based_on=row_embeddings'),
    ('tabpfn', 'device=cuda,predML_based_on=custom_predictiveML_model'),
    #('tabpfn', 'device=cuda,predML_based_on=row_embeddings'),
    ('tabula_8b', 'batch_size=1,device=cuda,max_length=512,model_name=mlfoundations_tabula-8b,n_few_shot_examples=10,predML_based_on=row_embeddings'),
]

In [32]:

def get_aggregated_metric(df, approach, config, metric_col, agg="mean"):
    """
    Aggregate a metric column across all datasets
    for a given approach/config.
    """
    sel = df[
        (df["Approach"] == approach) &
        (df["Configuration"] == config)
    ]

    if sel.empty or metric_col not in sel.columns:
        return np.nan

    if agg == "mean":
        return sel[metric_col].mean()
    elif agg == "median":
        return sel[metric_col].median()
    else:
        raise ValueError(f"Unknown aggregation: {agg}")

In [33]:
rows = []

for approach, config in include_runs:
    row = {
        "Method": approach,

        # Predictive ML (aggregated over datasets)
        "BinClass_AUC": get_aggregated_metric(
            predictive_ml_df,
            approach,
            config,
            "XGBoost_roc_auc_score (↑)_mean"
        ),

        "MultiClass_LogLoss": get_aggregated_metric(
            predictive_ml_df,
            approach,
            config,
            "XGBoost_log_loss (↓)_mean"
        ),

        "Regression_RMSE": get_aggregated_metric(
            predictive_ml_df,
            approach,
            config,
            "LinearRegression_rmse (↓)_mean"
        ),

        # Similarity search
        "RowSim_MRR": get_aggregated_metric(
            row_similarity_search_df,
            approach,
            config,
            "MRR_mean"
        ),

        "ColSim_MRR": get_aggregated_metric(
            column_similarity_search_df,
            approach,
            config,
            "MRR_mean"
        ),
    }

    rows.append(row)

table_df = pd.DataFrame(rows)

In [44]:
# rename column where 'method'==baseline to 'XGBoost'
table_df.loc[table_df['Method'] == 'baseline', 'Method'] = 'XGBoost'

# drop index col
table_df = table_df.reset_index(drop=True)

In [45]:
table_df

,Method,BinClass_AUC,MultiClass_LogLoss,Regression_RMSE,RowSim_MRR,ColSim_MRR
0,XGBoost,0.827346,0.312985,15233.901828,NaN,NaN
1,hytrel,0.697088,0.700025,30983.225607,0.089473,0.360285
2,sap_rpt_oss,0.825671,0.304046,7733.201571,0.072793,0.079489
3,sentence_transformer,0.720280,0.514160,20098.117343,0.660990,0.497374
4,tabicl,0.849730,0.253419,NaN,0.059022,0.002705
5,tabpfn,0.842889,0.272615,6620.207296,0.051020,NaN
6,tabula_8b,0.783890,0.435095,12828.181957,NaN,NaN


In [50]:
# get results_table as latex with formatting (two decimal places) and output written to file so that linebreaks are automatically handled, also add | between all columns
with open("overall_results.tex", "w") as f:
    f.write(table_df.to_latex(float_format="%.2f", column_format="|l|" + "c|" * (len(table_df.columns)), index=False))

In [51]:
# delete all cols that are nan 
row_similarity_search_df = row_similarity_search_df.dropna(axis=1, how='all')

In [52]:
row_similarity_search_df

,Approach,Configuration,task,dataset,# Runs,MRR_mean,In top-1 [%]_mean,In top-3 [%]_mean,In top-5 [%]_mean,In top-10 [%]_mean,...,model_setup---execution_time (s),model_setup---peak_cpu (%),model_setup---average_cpu (%),model_setup---peak_memory (MB),model_setup---average_memory (MB),task_inference---execution_time (s),task_inference---peak_cpu (%),task_inference---average_cpu (%),task_inference---peak_memory (MB),task_inference---average_memory (MB)
62,GritLM,"embedding_model=GritLM_GritLM-7B,table_row_lim...",row_similarity_search,Amazon-Google,1,0.569712,41.880342,66.666667,77.777778,90.170940,...,28.515242,1759.0,1266.907407,6433.816406,3712.460503,35.405647,4979.0,657.519209,2148.464844,1966.272842
63,GritLM,"embedding_model=GritLM_GritLM-7B,table_row_lim...",row_similarity_search,Beer,1,1.000000,100.000000,100.000000,100.000000,100.000000,...,5.775926,3565.9,2326.283333,6195.398438,3480.449219,53.571704,4109.3,154.085338,2174.003906,1988.296390
64,GritLM,"embedding_model=GritLM_GritLM-7B,table_row_lim...",row_similarity_search,DBLP-ACM,1,0.984047,97.297297,99.774775,100.000000,100.000000,...,5.880634,3509.6,2307.033333,6033.515625,3724.758464,76.412742,4838.9,628.232987,2186.000000,1986.681615
65,GritLM,"embedding_model=GritLM_GritLM-7B,table_row_lim...",row_similarity_search,DBLP-GoogleScholar,1,0.611445,41.588785,77.383178,88.411215,96.261682,...,5.803919,3483.3,2289.216667,6174.792969,3555.653646,1046.668880,2976.2,431.858719,5141.128906,2957.850637
66,GritLM,"embedding_model=GritLM_GritLM-7B,table_row_lim...",row_similarity_search,Fodors-Zagats,1,1.000000,100.000000,100.000000,100.000000,100.000000,...,5.835300,3500.2,2275.616667,6089.351562,3503.106771,7.757887,3524.8,217.164103,1949.140625,1898.405549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,tabula_8b,"batch_size=16,device=cuda,max_length=512,predM...",row_similarity_search,Fodors-Zagats,1,0.672727,59.090909,72.727273,72.727273,72.727273,...,9.665187,94.6,75.000000,5328.160156,3803.531250,46.515249,1038.6,107.260526,1542.617188,1492.458738
850,tabula_8b,"batch_size=16,device=cuda,max_length=512,predM...",row_similarity_search,MusicBrainz,1,0.225235,16.117333,20.010667,22.464000,26.197333,...,8.389385,96.1,78.800000,5323.457031,3796.454590,1985.310954,914.6,253.341357,2421.132812,1802.445936
851,tabula_8b,"batch_size=16,device=cuda,max_length=512,predM...",row_similarity_search,Walmart-Amazon,1,0.318569,19.689119,34.715026,43.523316,53.886010,...,9.093087,95.1,80.075000,2204.679688,1505.509277,1552.003045,901.9,110.579696,2680.195312,1711.783139
852,tabula_8b,"batch_size=16,device=cuda,max_length=512,predM...",row_similarity_search,geological-settlements,1,0.050000,0.000000,0.000000,0.000000,0.000000,...,9.184155,96.5,76.388889,4011.929688,2223.378472,184.989644,824.0,162.775138,1636.359375,1521.894310
